In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
from datetime import datetime
import matplotlib.pyplot as plt
import math
%matplotlib inline
import glob
import ast
from scipy.stats import chi2_contingency

In [2]:
exp_1_lines = pd.read_csv('/Users/tatyanach/Desktop/Cognitive Lab/9 dots/hypotheses_data/exp_1_lines_data.csv')  
exp_2_lines = pd.read_csv('/Users/tatyanach/Desktop/Cognitive Lab/9 dots/hypotheses_data/exp_2_lines_data.csv')
exp_3_lines = pd.read_csv('/Users/tatyanach/Desktop/Cognitive Lab/9 dots/hypotheses_data/exp_3_lines_data.csv')

In [3]:
exp_1_lines = exp_1_lines.set_index(['participant_id', 'try_id', 'line_id'], drop=False, append=False, inplace=False, verify_integrity=True)
exp_2_lines = exp_2_lines.set_index(['participant_id', 'try_id', 'line_id'], drop=False, append=False, inplace=False, verify_integrity=True)
exp_3_lines = exp_3_lines.set_index(['participant_id', 'try_id', 'line_id'], drop=False, append=False, inplace=False, verify_integrity=True)


In [4]:
exp_lines = pd.concat([exp_1_lines, exp_2_lines, exp_3_lines], keys = [0, 1, 2], names = ['exp_id'])
exp_lines

participant_id  try_id  line_id  \
exp_id participant_id try_id line_id                                    
0      0              0      0                     0       0        0   
                             1                     0       0        1   
                             2                     0       0        2   
                             3                     0       0        3   
       1              0      0                     1       0        0   
...                                              ...     ...      ...   
2      57             98     4                    57      98        4   
                      99     0                    57      99        0   
                             1                    57      99        1   
                             2                    57      99        2   
                             3                    57      99        3   

                                     dots_intersected             start  \
exp_id participant_id try_id line_id                                      
0      0              0      0                 [4, 2]  [5787.0, 3228.0]   
                             1              [2, 1, 0]  [6660.0, 4092.0]   
                             2                 [3, 7]  [4041.0, 4092.0]   
                             3              [8, 5, 2]  [6660.0, 1500.0]   
       1              0      0              [8, 4, 0]  [6660.0, 2364.0]   
...                                               ...               ...   
2      57             98     4              [2, 1, 0]  [6660.0, 4092.0]   
                      99     0                 [0, 1]  [4914.0, 4092.0]   
                             1                 [1, 4]  [5787.0, 4092.0]   
                             2                 [4, 5]  [5787.0, 3228.0]   
                             3                 [5, 2]  [6660.0, 3228.0]   

                                                   end  start_oob  end_oob  
exp_id participant_id try_id line_id                                        
0      0              0      0        [6660.0, 4092.0]          0        0  
                             1        [4041.0, 4092.0]          0        1  
                             2        [6660.0, 1500.0]          1        1  
                             3        [6660.0, 4092.0]          1        0  
       1              0      0        [4914.0, 4092.0]          0        0  
...                                                ...        ...      ...  
2      57             98     4        [4914.0, 4092.0]          0        0  
                      99     0        [5787.0, 4092.0]          0        0  
                             1        [5787.0, 3228.0]          0        0  
                             2        [6660.0, 3228.0]          0        0  
                             3        [6660.0, 4092.0]          0        0  

[40049 rows x 8 columns]

In [5]:
exp_1_solved = pd.read_csv('/Users/tatyanach/Desktop/Cognitive Lab/9 dots/hypotheses_data/exp_1_solved_data_clear.csv')  
exp_2_solved = pd.read_csv('/Users/tatyanach/Desktop/Cognitive Lab/9 dots/hypotheses_data/exp_2_solved_data_clear.csv')  
exp_3_solved = pd.read_csv('/Users/tatyanach/Desktop/Cognitive Lab/9 dots/hypotheses_data/exp_3_solved_data_clear.csv')  



In [6]:
exp_1_solved = exp_1_solved.set_index(['participant_id'], drop=False, append=False, inplace=False, verify_integrity=True)
exp_2_solved = exp_2_solved.set_index(['participant_id'], drop=False, append=False, inplace=False, verify_integrity=True)
exp_3_solved = exp_3_solved.set_index(['participant_id'], drop=False, append=False, inplace=False, verify_integrity=True)


In [7]:
exp_solved = pd.concat([exp_1_solved, exp_2_solved, exp_3_solved], keys = [0, 1, 2], names = ['exp_id'])

In [8]:
exp_solved

participant_id  solved
exp_id participant_id                        
0      0                            0       2
       1                            1       2
       2                            2       1
       3                            3       1
       4                            4       0
...                               ...     ...
2      53                          53       0
       54                          54       1
       55                          55       0
       56                          56       0
       57                          57       0

[177 rows x 2 columns]

In [9]:
side = ['side', {0, 1, 2}, {0, 3, 6}, {6, 7, 8}, {2, 5, 1}]
half_side = ['half_side', {0, 1}, {1, 2}, {2, 5}, {5, 8}, {8, 7}, {7, 6}, {6, 3}, {3, 0}]
main_diag = ['main_diag', {0, 4, 8}, {6, 4, 2}]
half_main_diag = ['half_main_diag', {0, 4}, {4, 8}, {2, 4}, {4, 6}]
middle_line = ['middle_line', {3, 4, 5}, {1, 4, 7}]
half_middle_line = ['half_middle_line', {3, 4}, {4, 5}, {1, 4}, {4, 7}]
side_diag = ['side_diag', {1, 3}, {3, 7}, {7, 5}, {5, 1}]
angle_to_side = ['angle_to_side', {0, 7}, {0, 5}, {2, 3}, {2, 7}, {8, 1}, {8, 3}, {6, 5}, {6, 1}]

line_types = [side, half_side, main_diag, half_main_diag, middle_line, half_middle_line, side_diag, angle_to_side]

In [10]:
res = exp_solved.copy()
res['half_lined_discovered'] = 0
res['oob_breakthrough'] = 0

for exp in range(3):
    #for each participant
    for p in range(max(exp_lines.loc[(exp), 'participant_id']) + 1):
        print('participant_id')
        print(p)
        discovered_lines_count = set()
        oob = 0
        breakthrough_search = -1000
        #for each try
        for t in range(max(exp_lines.loc[(exp, p), 'try_id']) + 1):
            #check if index is in range
            if (exp, p, t) in exp_lines.index:
                #for each line
                for l in range(len(exp_lines.loc[(exp, p, t), 'line_id'])):
                    #mark if it's out of the box
                    if (exp_lines.loc[(exp, p, t, l), 'start_oob'] == 1 
                    or exp_lines.loc[(exp, p, t, l), 'end_oob'] == 1):
                        oob = 1
                    #for each try    
                    for lt in line_types:
                        #see if the current line matches one of the types and if so, add it to 'discovered'
                        for q in range(1, len(lt)):
                            #print('in table')
                            #print(set(ast.literal_eval(exp_lines.loc[(exp, p, t, l), 'dots_intersected'])))
                            #print('in list')
                            #print(q)
                            present_line = set(ast.literal_eval(exp_lines.loc[(exp, p, t, l), 'dots_intersected']))
                            
                            if (len(present_line.difference(lt[q])) == 0
                            and len(lt[q].difference(present_line)) == 0):
                                discovered_lines_count.add(lt[0])
            #if we have any more breakthrough moves left
            if breakthrough_search > 0:
                breakthrough_search = breakthrough_search - 1
            
            #print('lines_discovered')
            #print(discovered_lines_count)
            
            #if the lines discovery condition is NOT met and we already have a breakthrough - we pass to the next participant
            if breakthrough_search == -1000 and oob == 1:
                break
            #if the lines discovery condition is met and we have 5 more moves than set breaktrough search for the 5
            elif breakthrough_search == -1000 and oob == 0 and len(discovered_lines_count) > 4 and t < max(exp_lines.loc[(exp, p), 'try_id']) - 5:
                res.loc[(exp, p), 'half_lined_discovered'] = 1 
                breakthrough_search = 5
            #if we search for a breakthrough and we find it -> check 1
            elif breakthrough_search > 0 and oob == 1:
                res.loc[(exp, p), 'oob_breakthrough'] = 1 
                break
            #if we search for a breakthrough and we don't find it within 5 moves -> check 0 (by default)
            elif breakthrough_search == 0:
                break
            

participant_id
0
participant_id
1
participant_id
2
lines_discovered
{'main_diag', 'side'}
lines_discovered
{'main_diag', 'half_side', 'side'}
lines_discovered
{'main_diag', 'half_side', 'middle_line', 'side'}
lines_discovered
{'middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'side_diag', 'half_main_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'half_middle_line', 'side_diag', 'half_main_diag', 'side', 'half_side'}
participant_id
3
lines_discovered
{'angle_to_side', 'side'}
participant_id
4
lines_discovered
{'main_diag'}
lines_discovered
{'middle_line', 'main_diag', 'angle_to_side', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'half_middle_line

lines_discovered
{'middle_line', 'main_diag', 'side_diag', 'angle_to_side', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'side_diag', 'angle_to_side', 'side', 'half_side'}
participant_id
13
lines_discovered
{'side'}
lines_discovered
{'main_diag', 'side'}
lines_discovered
{'main_diag', 'half_side', 'side'}
lines_discovered
{'main_diag', 'half_side', 'side'}
lines_discovered
{'main_diag', 'half_side', 'half_main_diag', 'side'}
lines_discovered
{'main_diag', 'half_side', 'half_main_diag', 'side'}
lines_discovered
{'middle_line', 'main_diag', 'half_main_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'half_middle_line', 'half_main_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'half_middle_line', 'half_main_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'half_middle_line', 'half_main_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'half_middle_line', 'half_main_di

lines_discovered
{'middle_line', 'main_diag', 'half_middle_line', 'half_main_diag', 'side', 'half_side'}
participant_id
25
lines_discovered
{'half_middle_line', 'half_side', 'side'}
lines_discovered
{'half_middle_line', 'half_side', 'side'}
lines_discovered
{'half_middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
participant_id
26
lines_discovered
{'side'}
lines_discovered
{'main_diag', 'side'}
lines_discovered
{'main_diag', 'middle_line', 'side', 'half_side'}
lines_discovered
{'main_

lines_discovered
{'middle_line', 'main_diag', 'half_middle_line', 'half_main_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'half_middle_line', 'half_main_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'half_middle_line', 'half_main_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'half_middle_line', 'half_main_diag', 'side', 'half_side'}
participant_id
39
lines_discovered
{'main_diag', 'half_side', 'side'}
lines_discovered
{'main_diag', 'side_diag', 'angle_to_side', 'side', 'half_side'}
lines_discovered
{'main_diag', 'side_diag', 'angle_to_side', 'side', 'half_side'}
lines_discovered
{'main_diag', 'side_diag', 'angle_to_side', 'side', 'half_side'}
lines_discovered
{'main_diag', 'side_diag', 'angle_to_side', 'side', 'half_side'}
lines_discovered
{'main_diag', 'side_diag', 'angle_to_side', 'side', 'half_side'}
lines_discovered
{'main_diag', 'side_diag', 'angle_to_side', 'half_main_diag', 'side', 'half_side'}


lines_discovered
{'middle_line', 'main_diag', 'half_middle_line', 'half_main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'half_middle_line', 'half_main_diag', 'side_diag', 'side', 'half_side'}
participant_id
11
lines_discovered
{'side', 'side_diag', 'half_side', 'middle_line'}
lines_discovered
{'side', 'side_diag', 'half_side', 'middle_line'}
lines_discovered
{'side', 'side_diag', 'half_side', 'middle_line'}
lines_discovered
{'middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
participant_id
12
lines_discovered
{'side_diag', 'half_main_diag'

...

lines_discovered
{'middle_line', 'main_diag', 'half_main_diag', 'angle_to_side', 'side'}
lines_discovered
{'middle_line', 'main_diag', 'half_main_diag', 'angle_to_side', 'side'}
lines_discovered
{'middle_line', 'main_diag', 'half_main_diag', 'angle_to_side', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'half_middle_line', 'half_main_diag', 'angle_to_side', 'side', 'half_side'}
participant_id
5
lines_discovered
{'side'}
lines_discovered
{'side'}
lines_discovered
{'side'}
lines_discovered
{'side'}
lines_discovered
{'main_diag', 'side'}
lines_discovered
{'main_diag', 'side_diag', 'side'}
lines_discovered
{'middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
participant_id
6
lines_discovered
set()
lines_discovered
{'side'}
participant_id
7
lines_discovered
{'main_diag', 'angle_to_side'}
lines_discovere

lines_discovered
{'half_middle_line', 'half_side', 'middle_line', 'side'}
lines_discovered
{'half_middle_line', 'half_side', 'middle_line', 'side'}
lines_discovered
{'half_middle_line', 'half_side', 'middle_line', 'side'}
lines_discovered
{'middle_line', 'half_middle_line', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
participant_id
19
lines_discovered
{'side_diag', 'main_diag', 'half_side'}
lines_discovered
{'side_diag', 'main_diag', 'half_side', 'side'}
participant_id
20
lines_discovered
{'side'}
lines_discovered
{'m

lines_discovered
{'middle_line', 'half_side', 'side'}
lines_discovered
{'middle_line', 'half_side', 'half_main_diag', 'side'}
lines_discovered
{'middle_line', 'half_middle_line', 'half_main_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'half_main_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'half_main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'half_main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'half_main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'half_main_diag', 'side_diag', 'side', 'half_side'}
participant_id
32
lines_discovered
{'main_diag', 'half_side', 'angle_to_side'}
lines_discovered
{'main_diag', 'half_side', 'side', 'angle_to_side'}
lines_discovered
{'middle_line', 'main_diag', 'angle_to_side', 'side',

lines_discovered
{'side_diag', 'half_side', 'half_middle_line'}
lines_discovered
{'side_diag', 'half_side', 'half_middle_line', 'side'}
lines_discovered
{'middle_line', 'half_middle_line', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'half_main_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'half_main_diag', 'angle_to_side', 'side', 'half_side'}
participant_id
43
lines_discovered
{'main_diag', 'side'}
lines_discovered
{'main_diag', 'side'}
participant_id
44
lines_discovered
{'side_diag', 'main_diag', 'half_side', 'side'}
lines_discovered
{'side_diag', 'main_diag', 'half_side', '

lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'half_main_diag', 'side', 'half_side'}
participant_id
56
lines_discovered
set()
lines_discovered
{'half_main_diag', 'half_middle_line', 'side'}
lines_discovered
{'half_main_diag', 'half_middle_line', 'side'}
lines_discovered
{'half_middle_line', 'main_diag', 'side_diag', 'half_main_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'half_main_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'half_main_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'half_main_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'half_main_diag', 'side', 'half_side'}
lines_discovered
{'middle_line', 'half_middle_line', 'main_diag', 'side_diag', 'half_main_diag', 'side', 'half_side'}


In [11]:
res

participant_id  solved  half_lined_discovered  \
exp_id participant_id                                                  
0      0                            0       2                      0   
       1                            1       2                      0   
       2                            2       1                      1   
       3                            3       1                      0   
       4                            4       0                      1   
...                               ...     ...                    ...   
2      53                          53       0                      1   
       54                          54       1                      0   
       55                          55       0                      1   
       56                          56       0                      1   
       57                          57       0                      0   

                       oob_breakthrough  
exp_id participant_id                    
0      0                              0  
       1                              0  
       2                              0  
       3                              0  
       4                              0  
...                                 ...  
2      53                             0  
       54                             0  
       55                             0  
       56                             0  
       57                             0  

[177 rows x 4 columns]

In [12]:
#drop non-valid participants
res = res.loc[res['solved'] != 2]
res

participant_id  solved  half_lined_discovered  \
exp_id participant_id                                                  
0      2                            2       1                      1   
       3                            3       1                      0   
       4                            4       0                      1   
       5                            5       0                      1   
       6                            6       0                      1   
...                               ...     ...                    ...   
2      53                          53       0                      1   
       54                          54       1                      0   
       55                          55       0                      1   
       56                          56       0                      1   
       57                          57       0                      0   

                       oob_breakthrough  
exp_id participant_id                    
0      2                              0  
       3                              0  
       4                              0  
       5                              0  
       6                              0  
...                                 ...  
2      53                             0  
       54                             0  
       55                             0  
       56                             0  
       57                             0  

[154 rows x 4 columns]

In [13]:
res.loc[res['half_lined_discovered'] == 1]

participant_id  solved  half_lined_discovered  \
exp_id participant_id                                                  
0      2                            2       1                      1   
       4                            4       0                      1   
       5                            5       0                      1   
       6                            6       0                      1   
       7                            7       0                      1   
...                               ...     ...                    ...   
2      48                          48       0                      1   
       50                          50       0                      1   
       53                          53       0                      1   
       55                          55       0                      1   
       56                          56       0                      1   

                       oob_breakthrough  
exp_id participant_id                    
0      2                              0  
       4                              0  
       5                              0  
       6                              0  
       7                              0  
...                                 ...  
2      48                             0  
       50                             0  
       53                             0  
       55                             0  
       56                             0  

[105 rows x 4 columns]

In [14]:
res.loc[res['oob_breakthrough'] == 1]

participant_id  solved  half_lined_discovered  \
exp_id participant_id                                                  
0      8                            8       1                      1   
       22                          22       0                      1   
1      17                          17       1                      1   
       31                          31       0                      1   
2      5                            5       1                      1   
       25                          25       0                      1   

                       oob_breakthrough  
exp_id participant_id                    
0      8                              1  
       22                             1  
1      17                             1  
       31                             1  
2      5                              1  
       25                             1

In [15]:
def make_contingency_table(test_data, groups, values):
    cols = groups + values
    df = test_data.loc[:, cols]
    df = df.groupby(groups).count()
    #fill non-existant cases
    if (0,0) not in df.index:
        df.loc[(0,0), df.columns[0]] = 0
    if (0,1) not in df.index:
        df.loc[(0,1), df.columns[0]] = 0
    if (1,0) not in df.index:
        df.loc[(1,0), df.columns[0]] = 0
    if (1,1) not in df.index:
        df.loc[(1,1), df.columns[0]] = 0
    res = [[int(df.loc[(0, 0)]), int(df.loc[(0, 1)])], [int(df.loc[(1, 0)]), int(df.loc[(1, 1)])]]
    return res

In [16]:
groups = ['half_lined_discovered', 'oob_breakthrough']
values = ['participant_id']
df = make_contingency_table(res, groups, values)
print(groups)
print('0/0, 0/1, 1/0, 1/1')
print(df)
check_sum = df[0][0] + df[0][1] + df[1][0] + df[1][1]
print('check_sum')
print(check_sum)

['half_lined_discovered', 'oob_breakthrough']
0/0, 0/1, 1/0, 1/1
[[49, 0], [99, 6]]
check_sum
154


In [17]:
def test_me(test_data, groups, values):
    print('Test factors:')
    print(groups)
    obs = make_contingency_table(test_data, groups, values)
    print('contingency table:')
    print(groups)
    print('0/0, 0/1, 1/0, 1/1')
    print(obs)
    chi2, pvalue, dof, expectedndarray = chi2_contingency(obs)
    #Returns: test statistic, p-value, degrees of freedom, expected frequencies
    print('chi2-criterium is')
    print(chi2)
    print('p-value is:')
    print(pvalue)
    if pvalue < 0.05:
        return 'REJECT the null hypothesis - The factors are DEPENDENT!\n'
    else:
        return 'Cant reject the null hypothesis - The factors are independent\n'

In [18]:
#testing on merged data (exp 1-3)
df = res
groups = ['half_lined_discovered', 'oob_breakthrough']
values = ['participant_id']
print(test_me(df, groups, values))

Test factors:
['half_lined_discovered', 'oob_breakthrough']
contingency table:
['half_lined_discovered', 'oob_breakthrough']
0/0, 0/1, 1/0, 1/1
[[49, 0], [99, 6]]
chi2-criterium is
1.5872372372372372
p-value is:
0.20772130003193132
Cant reject the null hypothesis - The factors are independent

